# 1.Original model prediction

In [1]:
from train import *
from scipy.special import expit
import argparse
def predict(cdrs,epitopes,labels,model,batch_size = 128):
    batch_num_total = len(cdrs) // batch_size if len(cdrs) % batch_size == 0 else len(cdrs) // batch_size + 1
    y_pres,y_trues ,epitope,y_preds_binary,CDR3B= [],[],[],[],[]
    for batch_num in tqdm(range(batch_num_total)):
        end = (batch_num+1)*batch_size if (batch_num+1)*batch_size <= len(cdrs) else len(cdrs)                
        ts,es = cdrs[batch_num*batch_size:end],epitopes[batch_num * batch_size:end] 
        score = model(ts,es) 
        y_pres.extend(expit(score.view(-1).detach().cpu().numpy()))
        y_trues.extend(labels[batch_num*batch_size : end])
        epitope.extend(epitopes[batch_num*batch_size: end])
        CDR3B.extend(cdrs[batch_num*batch_size: end])
        threshold = 0.5
        y_preds_binary = [1 if p >= threshold else 0 for p in y_pres]
        df = pd.DataFrame({'epitope':epitope,'CDR3B': CDR3B, 'y_true': y_trues,'y_pred':y_preds_binary, 'y_prob': y_pres})
        #name_str = '_'.join(name)
        result_path1 = result_path +  'probability.csv'
        pd.DataFrame(df).to_csv(result_path1)   
    return {'epitope': epitope,'CDR3B': CDR3B,'y_true':y_trues,'y_prob':y_pres,'y_pred':y_preds_binary}

class Args:
    use_column='CDR3B'
    batch_size=128
  #  modelfile_path=modelfile_path
    device='cuda:0'
    def __init__(self, testfile_path, result_path,modelfile_path):
        self.testfile_path = testfile_path 
        self.result_path = result_path
        self.modelfile_path = modelfile_path
        
def Original_model_prediction(testfile_path,modelfile_path,result_path):
    import pandas as pd
    from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, f1_score, matthews_corrcoef, precision_score,auc,roc_curve
    args = Args(testfile_path=testfile_path, result_path=result_path, modelfile_path=modelfile_path)
    f = pd.read_csv(args.testfile_path)
    #load model
    model_tcr = TCRpeg(hidden_size=768,num_layers = 3,load_data=False,embedding_path='encoders/aa_emb_tcr.txt',device=args.device)
    model_tcr.create_model()
    model_epi = TCRpeg(hidden_size=768,num_layers = 3,load_data=False,embedding_path='encoders/aa_emb_tcr.txt',device=args.device)
    model_epi.create_model()
    model = TEINet(en_tcr=model_tcr,en_epi = model_epi,cat_size=768*2,dropout=0.1,normalize=True,weight_decay = 0,device=args.device).to(args.device)
    model.load_state_dict(torch.load(args.modelfile_path))
    # model = torch.load(args.modelfile_path,map_location='cuda:0')
    cdrs,epitopes,labels = f['CDR3B'].values, f['Epitope'].values, f['Affinity'].values
    pres = predict(cdrs,epitopes,labels,model)
    result = pd.DataFrame(pres)
    result.to_csv(result_path+'probability.csv')

In [ ]:
testfile_path="../data/test.csv"
modelfile_path="../Original_model/TEINet_large.pth"
result_path="../result_path/Original_model_prediction"
Original_model_prediction(testfile_path,modelfile_path,result_path)

In [ ]:
testfile_path="../data/test.csv"
modelfile_path="../Original_model/TEINet_small.pth"
result_path="../result_path/Original_model_prediction"
Original_model_prediction(testfile_path,modelfile_path,result_path)

# 2.Model retraining

In [3]:
from tcrpeg.TCRpeg import TCRpeg
import pandas as pd
import numpy as np
import argparse
from sklearn.metrics import roc_auc_score as AUC
from sklearn.metrics import average_precision_score as AUPRC
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from sampler import Sampler
from model import TEINet
import psutil
import time
class Args:
    dropout=0.0
    epochs=3
    cat_size=1536
    batch_size=48
    sample_num=10
    info='information'
    aa_tcr='encoders/aa_emb_tcr.txt'
    aa_epi= 'encoders/aa_emb_epi.txt'
    pretrain_tcr='encoders/encoder_tcr.pth'
    pretrain_epi='encoders/encoder_epi.pth'
    weight=4
    weight_decay=0.0
    normalize=1  
    step1=21
    step2=27
    step3=30          
    lr=0.001              
    record_path='results/predictions.txt'   
   # output_path='results/training.txt'
  #  save_modle_path='None'
   # trainfile_path='data/train_pos.csv'
    static=0
   # test_file='None'
    pretrain=True
   # sample_strategy='sample_epi'
    def __init__(self,result_path, trainfile_path,save_modle_path,testfile_path):
        self.result_path = result_path
        self.trainfile_path = trainfile_path
        self.testfile_path=testfile_path
        self.save_modle_path=save_modle_path

def Model_retraining(trainfile_path,testfile_path,save_modle_path,result_path):
    import pandas as pd
    from sklearn.utils import shuffle
    args = Args(result_path=result_path, trainfile_path=trainfile_path,save_modle_path=save_modle_path,testfile_path=testfile_path)  
    normalize = False if args.normalize == 0 else True     
    hidden_size = args.cat_size // 2 
    cat_size = args.cat_size        
    static = True if args.static == 1 else False
    model_tcr = TCRpeg(hidden_size=hidden_size,num_layers = 3,load_data=False,embedding_path=args.aa_tcr)
    #load pretrained TCR model
    if args.pretrain: 
        print('using the pretrained model')
        model_tcr.create_model(load=True,path=args.pretrain_tcr)
    else :
        model_tcr.create_model()
    model_tcr.model.train()
    model_epi = TCRpeg(hidden_size=hidden_size,num_layers = 3,load_data=False,embedding_path=args.aa_epi)
    if args.pretrain:                             
        model_epi.create_model(load=True,path = args.pretrain_epi)
    else :
        model_epi.create_model()
    model_epi.model.train()
    dropout = args.dropout
    cat_size = args.cat_size
    model = TEINet(en_tcr=model_tcr,en_epi = model_epi,cat_size=cat_size,dropout=dropout,normalize=normalize,weight_decay = args.weight_decay).to('cuda:0')
    optimizer = optim.Adam(model.parameters(), lr=args.lr)
    if args.testfile_path != 'None':
        data = pd.read_csv(args.testfile_path,low_memory=False)   
        cs_test,es_test,ls_test = data['CDR3B'].values,data['Epitope'].values,data['Affinity'].values
    pos_weight = (torch.ones([1])*args.weight).to('cuda:0')    
    loss_fcn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    epochs = args.epochs

    batch_size = args.batch_size
    sample_num = args.sample_num      
    record_aucs = []
    train_data = pd.read_csv(args.trainfile_path,low_memory=False)   
    cs_train,es_train,ls_train = train_data['CDR3B'].values,train_data['Epitope'].values,train_data['Affinity'].values
    tcrs, epitopes, labels = shuffle(cs_train, es_train, ls_train, random_state=123)
    infer = np.random.permutation(len(tcrs))
    tcrs, epitopes, labels = tcrs[infer], epitopes[infer], labels[infer]
    for e in range(epochs):
        batch_num_total = len(tcrs) // batch_size if len(tcrs) % batch_size == 0 else len(tcrs) // batch_size + 1
        infer = np.random.permutation(len(tcrs))
        for batch_num in tqdm(range(batch_num_total)):
            end = (batch_num+1)*batch_size if (batch_num+1)*batch_size <= len(tcrs) else len(tcrs)                
            ts,es = tcrs[batch_num*batch_size:end],epitopes[batch_num * batch_size:end]             
            ls = labels[batch_num*batch_size:end]                
            ls = torch.FloatTensor(ls).to('cuda:0')  
            output = model(ts,es)
            if args.weight_decay == 0.0:
                loss = loss_fcn(output.view(-1),ls)
            else :
                loss = loss_fcn(output[0].view(-1),ls) + output[1]   
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        y_pres = []
        y_trues = []
        CDR3B = []
        epitope = []
        y_preds_binary =[]
        if args.testfile_path != 'None':
            with torch.no_grad():
                y_pres = []
                y_trues = []
                batch_num = len(cs_test) // batch_size if len(cs_test) % batch_size == 0 else len(cs_test) // batch_size + 1        
                for i in tqdm(range(batch_num)):
                    end = (i+1)*batch_size if (i+1)*batch_size <= len(cs_test) else len(cs_test)                
                    cs_batch,es_batch = cs_test[i*batch_size :end], es_test[i*batch_size : end]
                    score = model(cs_batch,es_batch) 
                    if args.weight_decay !=0.0:
                        score = score[0]           
                    y_pres.extend(score.view(-1).detach().cpu().numpy())  
                    y_trues.extend(ls_test[i*batch_size : end])
                    epitope.extend(es_test[i*batch_size : end])
                    CDR3B.extend(cs_test[i*batch_size : end])
                y_pres = np.array(y_pres)
                y_pres = 1 / (1 + np.exp(-y_pres))
                threshold = 0.5
                y_preds_binary = [1 if p >= threshold else 0 for p in y_pres]
                df = pd.DataFrame({'Epitope':epitope,'CDR3B': CDR3B, 'y_true': y_trues,'y_pred':y_preds_binary, 'y_prob': y_pres})
                pd.DataFrame(df).to_csv(result_path+'probability.csv')   
                test_auc = AUC(y_trues,y_pres)            
                print('Epoch: ',e)
                print('Test AUC: ',AUC(y_trues,y_pres))
                record_aucs.append(AUC(y_trues,y_pres))
        if e == args.step1 or e == args.step2:
            for g in optimizer.param_groups:
                g['lr'] = g['lr'] * 0.01
            print('change the learning rate to 1e-4')
        if e == args.step3:
            for g in optimizer.param_groups:
                g['lr'] = g['lr'] * 0.02
            print('change')
    if args.save_modle_path != 'None':
        if not args.save_modle_path.endswith('.pth'):
            args.save_modle_path = args.save_modle_path 
        torch.save(model.state_dict(),args.save_modle_path)


When retraining the model, you need to download the pre-trained `encoder_epi.pth` and `encoder_tcr.pth` from this link: https://github.com/jiangdada1221/TEINet/tree/master/encoders, and place them in the `encoders` folder.

In [ ]:
trainfile_path ="../data/train.csv"
testfile_path="../data/test.csv"
save_modle_path="../Retraining_model/Retraining_model.pth"
result_path="../result_path/Retraining_model_prediction"
Model_retraining(trainfile_path,testfile_path,save_modle_path,result_path) 

# 3.Retraining_model_prediction

In [12]:
from train import *
from scipy.special import expit
import argparse
def predict(cdrs,epitopes,labels,model,batch_size = 128):
    batch_num_total = len(cdrs) // batch_size if len(cdrs) % batch_size == 0 else len(cdrs) // batch_size + 1
    y_pres,y_trues ,epitope,y_preds_binary,CDR3B= [],[],[],[],[]
    for batch_num in tqdm(range(batch_num_total)):
        end = (batch_num+1)*batch_size if (batch_num+1)*batch_size <= len(cdrs) else len(cdrs)                
        ts,es = cdrs[batch_num*batch_size:end],epitopes[batch_num * batch_size:end] 
        score = model(ts,es) 
        y_pres.extend(expit(score.view(-1).detach().cpu().numpy()))
        y_trues.extend(labels[batch_num*batch_size : end])
        epitope.extend(epitopes[batch_num*batch_size: end])
        CDR3B.extend(cdrs[batch_num*batch_size: end])
        threshold = 0.5
        y_preds_binary = [1 if p >= threshold else 0 for p in y_pres]
        df = pd.DataFrame({'epitope':epitope,'CDR3B': CDR3B, 'y_true': y_trues,'y_pred':y_preds_binary, 'y_prob': y_pres})
        #name_str = '_'.join(name)
        result_path1 = result_path +  'probability.csv'
        pd.DataFrame(df).to_csv(result_path1)   
    return {'epitope': epitope,'CDR3B': CDR3B,'y_true':y_trues,'y_prob':y_pres,'y_pred':y_preds_binary}

class Args:
    use_column='CDR3B'
    batch_size=128
  #  modelfile_path=modelfile_path
    device='cuda:0'
    def __init__(self, testfile_path, result_path,modelfile_path):
        self.testfile_path = testfile_path 
        self.result_path = result_path
        self.modelfile_path = modelfile_path
        
def Retraining_model_prediction(testfile_path,modelfile_path,result_path):
    import pandas as pd
    from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, f1_score, matthews_corrcoef, precision_score,auc,roc_curve
    args = Args(testfile_path=testfile_path, result_path=result_path, modelfile_path=modelfile_path)
    f = pd.read_csv(args.testfile_path)
    #load model
    model_tcr = TCRpeg(hidden_size=768,num_layers = 3,load_data=False,embedding_path='encoders/aa_emb_tcr.txt',device=args.device)
    model_tcr.create_model()
    model_epi = TCRpeg(hidden_size=768,num_layers = 3,load_data=False,embedding_path='encoders/aa_emb_tcr.txt',device=args.device)
    model_epi.create_model()
    model = TEINet(en_tcr=model_tcr,en_epi = model_epi,cat_size=768*2,dropout=0.1,normalize=True,weight_decay = 0,device=args.device).to(args.device)
    model.load_state_dict(torch.load(args.modelfile_path))
    # model = torch.load(args.modelfile_path,map_location='cuda:0')
    cdrs,epitopes,labels = f['CDR3B'].values, f['Epitope'].values, f['Affinity'].values
    pres = predict(cdrs,epitopes,labels,model)
    result = pd.DataFrame(pres)
    result.to_csv(result_path+'probability.csv')

In [ ]:
testfile_path="../data/Validation.csv"
modelfile_path="../Retraining_model/Retraining_model.pth"
result_path="../result_path/Retraining_model_prediction"
Retraining_model_prediction(testfile_path,modelfile_path,result_path)
